In [19]:
import subprocess
from itertools import product
from math import prod
from random import randrange

import numpy as np
import pandas as pd
from multiprocess import Pool

## Варіант 5

S = (F, 8, E, 9, 7, 2, 0, D, C, 6, 1, 5, B, 4, 3, A)

In [26]:
s_block = [0xF, 0x8, 0xE, 0x9, 0x7, 0x2, 0x0, 0xD, 0xC, 0x6, 0x1, 0x5, 0xB, 0x4, 0x3, 0xA]
s_block_inverse = [0x6, 0xA, 0x5, 0xE, 0xD, 0xB, 0x9, 0x4, 0x1, 0x3, 0xF, 0xC, 0x8, 0x7, 0x2, 0x0]
ENC = True
DEC = False
MAX_16BIT_NUM = (1 << 16) - 1
VARIANT = 5


def get_hex(l):
    return '[{}]'.format(', '.join(hex(x) for x in l))


def get_hex_tuple_0(l):
    return '[{}]'.format(', '.join((hex(x[0])) for x in l))


class heys:

    def heys_round(self, n, key=0):
        ct = n ^ key
        ct = self.substitute(ct, s_block)
        ct = self.mix_words(ct)
        return ct

    # r = (s_1(y_1), s_2(y_2), ... , s_n(y_n)) 
    def substitute(self, n, enc):
        s = s_block if enc == True else s_block_inverse
        r = 0
        for i in range(4):
            r |= s[(n >> (i * 4)) & 15] << (i * 4)
        return r

    # i-тий біт j-того фрагменту стає j-тим бітом i-того фрагменту.
    def mix_words(self, n):
        r = 0
        for j in range(4):
            for i in range(4):
                r |= (n >> (4 * j + i) & 1) << (4 * i + j)
        return r


## Створення таблиці лінійних потенціалів

In [27]:
heys_obj = heys()
# таблиця лінійних потенціалів для перестановки шифру Хейса
linear_potential = np.zeros((16, 16))

# makes AND operation to get what bits has to be XORed and performs XOR of all 
#  variables by taking weight of number and mod 2
def custom_mul(x, y):
    return (x & y).bit_count() % 2

for alpha in range(16):
    for beta in range(16):
        for x in range(16):
            linear_potential[alpha][beta] += (-1) ** (custom_mul(alpha, x) ^ custom_mul(beta, heys_obj.substitute(x, ENC)))
linear_potential /= 16
linear_potential **= 2

pd.DataFrame(linear_potential)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.0,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.2500,0.0625,0.0625,0.0000
2,0.0,0.0625,0.0625,0.2500,0.0625,0.0000,0.0000,0.0625,0.0000,0.0625,0.0625,0.2500,0.0625,0.0000,0.0000,0.0625
3,0.0,0.2500,0.0000,0.0000,0.0625,0.0625,0.0625,0.0625,0.2500,0.0000,0.0000,0.0000,0.0625,0.0625,0.0625,0.0625
4,0.0,0.0000,0.0625,0.0625,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.0000,0.2500,0.0000,0.0000,0.0625,0.0625
5,0.0,0.0625,0.0000,0.0625,0.0625,0.0000,0.0625,0.2500,0.0625,0.0000,0.0625,0.0000,0.2500,0.0625,0.0000,0.0625
6,0.0,0.0625,0.0000,0.0625,0.0000,0.0625,0.0000,0.0625,0.0625,0.0000,0.0625,0.0000,0.0625,0.2500,0.0625,0.2500
7,0.0,0.0000,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.2500,0.2500
8,0.0,0.0000,0.0000,0.0000,0.0000,0.2500,0.2500,0.0000,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625
9,0.0,0.0625,0.0625,0.0000,0.2500,0.0625,0.0625,0.0000,0.0625,0.0000,0.2500,0.0625,0.0625,0.0000,0.0000,0.0625


In [28]:
PROBABILITY_THRESHOLD = 0.00015
local_diff_table = {}


# betas
def get_new_betas(alpha):
    # obtain raw alpha values from one big Alpha
    alpha_list = [(alpha >> (4 * i)) & 15 for i in range(4)]
    # obtain non-null betas from s block diff table
    non_null_betas = []
    for i, alpha in enumerate(alpha_list):
        # filter betas with non-zero entries
        betas = [beta for beta in range(16) if linear_potential[alpha][beta] != 0]
        non_null_betas.append(betas)

    betas = {}
    # generate all possible combinations that can appear in resulted beta
    for (i, beta_list) in enumerate(product(*non_null_betas, repeat=1)):
        beta = sum([beta_list[i] << (i * 4) for i in range(4)])
        betas[beta] = prod([linear_potential[alpha_list[i]][beta_list[i]] for i in range(4)])

    return betas


def differential_search(alpha):
    # shared between workers variable
    global local_diff_table

    # Г_{t-1}(alpha) (Г spells like 'g')
    g_prev = {alpha: 1}
    # like in rust -- 1..=5
    for i in range(1, 6):
        g_current = {}
        # misuse of 'beta' naming (by now we are iterating over last alphas right before calculation of probabilities)
        for beta, p_i in g_prev.items():

            if beta not in local_diff_table:
                local_diff_table[beta] = get_new_betas(beta)

            # extract possible candidates for specific beta ('alpha') and iterate over them
            for gamma in local_diff_table[beta]:
                tmp_prob = p_i * local_diff_table[beta][gamma]
                if gamma not in g_current:
                    g_current[gamma] = tmp_prob
                else:
                    g_current[gamma] += tmp_prob

        # Г_{t}(alpha)
        g_new_filtered = {}
        #check "bounds" and write to new list mixed words (aka gammas are becoming new alphas)
        for gamma in g_current.keys():
            if g_current[gamma] > PROBABILITY_THRESHOLD:
                g_new_filtered[heys_obj.mix_words(gamma)] = g_current[gamma]

        g_prev = g_new_filtered

    return (alpha, g_prev)

## Знайдені за допомогою методу «гілок та границь» високоймовірні диференціали
Вони наведені у формі $(\alpha \rightarrow [(\beta, prob)  \dots ], \dots )$

In [29]:
%%time

def prepare_alphas():
    alfas = []
    for i in range(4):
        for j in range(1, 16):
            alfas.append(j << (4 * i))
    return alfas


def init_worker(shared_diff_table):
    global local_diff_table
    local_diff_table = shared_diff_table


with Pool(initializer=init_worker, initargs=(local_diff_table,)) as pool:
    candidates = list(
        pool.map(
            differential_search,
            prepare_alphas()
        )
    )

print('Possible candidates after differential search: ')
list(candidates)

Possible candidates after differential search: 
CPU times: user 384 ms, sys: 122 ms, total: 507 ms
Wall time: 1min 23s


[(1,
  {17: 0.00018454398377798498,
   257: 0.00023846482508815825,
   272: 0.00026313314447179437,
   4352: 0.00019629453890956938,
   4096: 0.00016126653645187616,
   136: 0.0002424493432044983,
   2056: 0.0003714263439178467,
   2176: 0.00040375441312789917,
   34816: 0.000280916690826416,
   34944: 0.00020729750394821167,
   32904: 0.00019574910402297974,
   32768: 0.0002263486385345459}),
 (2,
  {17: 0.0001951456069946289,
   257: 0.00017428936553187668,
   272: 0.0002173835237044841,
   4352: 0.00019179822993464768,
   136: 0.00024132616817951202,
   2056: 0.0002853255718946457,
   2176: 0.00031377002596855164,
   34816: 0.0002876501530408859,
   34824: 0.00016975030303001404,
   34944: 0.00018735788762569427,
   32768: 0.00019478239119052887}),
 (3,
  {1: 0.00017478116205893457,
   17: 0.0001899251656141132,
   257: 0.000169116334291175,
   4352: 0.0002463887503836304,
   4353: 0.00020746083464473486,
   4368: 0.00017448668950237334,
   273: 0.0001525094558019191,
   4369: 0.000

## Відфільтровані кандидати 
Вони наведені у формі $((\alpha, \beta), prob)$

In [20]:
filtered_candidates = []
for alpha, betas in list(candidates):
    for beta, prob in betas.items():
        filtered_candidates.append(((alpha, beta), prob))
filtered_candidates = sorted(filtered_candidates, key=lambda x: x[1], reverse=True)
filtered_candidates

[((1024, 273), 0.00206756591796875),
 ((3840, 273), 0.0017242431640625),
 ((3072, 273), 0.001537322998046875),
 ((1024, 2184), 0.0014445781707763672),
 ((2560, 273), 0.001422882080078125),
 ((3072, 4369), 0.001422882080078125),
 ((3072, 4368), 0.001346588134765625),
 ((1024, 4369), 0.00133514404296875),
 ((3840, 2184), 0.0012636184692382812),
 ((3072, 1), 0.001178741455078125),
 ((1024, 17), 0.0011653900146484375),
 ((3072, 17), 0.0011539459228515625),
 ((4, 546), 0.0011348724365234375),
 ((3072, 4096), 0.0011348724365234375),
 ((1024, 546), 0.0011004209518432617),
 ((3584, 273), 0.001068115234375),
 ((3072, 16), 0.0010585784912109375),
 ((1792, 273), 0.0010528564453125),
 ((1024, 34952), 0.001047372817993164),
 ((3840, 4369), 0.00102996826171875),
 ((3328, 4369), 0.0010166168212890625),
 ((3328, 273), 0.00101470947265625),
 ((768, 273), 0.0009918212890625),
 ((3072, 2184), 0.0009708404541015625),
 ((3072, 34944), 0.0009708404541015625),
 ((3072, 34952), 0.0009708404541015625),
 ((2560

## Перевірка ключів

In [21]:
%%time

SAMPLES_NUM = 7000


# text -- 16bit numbers
def write_file(file, text: list[int]):
    with open(file, 'wb') as f:
        for x in text:
            f.write(x.to_bytes(2, byteorder='little'))
        f.close()


def read_file(file):
    text = []
    with open(file, 'rb') as f:
        data = f.read()
        for pair in zip(data[::2], data[1::2]):
            el1, el2 = pair
            text.append((el2 << 8) | el1)
        f.close()
    return text


def get_plaintext_filename(variant: int, index: int, is_mutated: bool):
    return './key_check/pt_{0}_{1}{2}.bin'.format(variant, index, '_mut' if is_mutated else '')


def get_ciphertext_filename(variant: int, index: int, is_mutated: bool):
    return './key_check/ct_{0}_{1}{2}.bin'.format(variant, index, '_mut' if is_mutated else '')


def generate_ciphertext(alpha, samples_num, index):
    plaintext = [randrange(0, MAX_16BIT_NUM) for _ in range(0, samples_num)]
    mutated_plaintext = [text ^ alpha for text in plaintext]

    filename_pt_nonmut = get_plaintext_filename(VARIANT, index, False)
    filename_pt_mut = get_plaintext_filename(VARIANT, index, True)
    filename_ct_nonmut = get_ciphertext_filename(VARIANT, index, False)
    filename_ct_mut = get_ciphertext_filename(VARIANT, index, True)
    write_file(filename_pt_nonmut, plaintext)
    write_file(filename_pt_mut, mutated_plaintext)

    subprocess.Popen('./data/heys.bin e {0} {1} {2}'.format(VARIANT, filename_pt_nonmut, filename_ct_nonmut),
                     shell=True,
                     stdout=subprocess.DEVNULL, stderr=None).communicate(timeout=10)
    subprocess.Popen('./data/heys.bin e {0} {1} {2}'.format(VARIANT, filename_pt_mut, filename_ct_mut), shell=True,
                     stdout=subprocess.DEVNULL,
                     stderr=None).communicate(timeout=10)

    ciphertext = read_file(filename_ct_nonmut)
    ciphertext_mut = read_file(filename_ct_mut)
    return ciphertext, ciphertext_mut


def check_candidate(ct_nonmut, ct_mut, k, beta):
    success_rate = 0
    for (ct1, ct2) in zip(ct_nonmut, ct_mut):
        d1 = heys_obj.substitute(heys_obj.mix_words(ct1 ^ k), DEC)
        d2 = heys_obj.substitute(heys_obj.mix_words(ct2 ^ k), DEC)
        if d1 ^ d2 == beta:
            success_rate += 1
    return success_rate


# checking keys from 0..=((1<<16)-1)
keys = [(i, 0) for i in range(0, MAX_16BIT_NUM + 1)]

for i, ((alpha, beta), _) in enumerate(filtered_candidates):
    ct_nonmut, ct_mut = generate_ciphertext(alpha, SAMPLES_NUM, i)

    with Pool() as pool:
        success_rates = list(pool.map(lambda k: check_candidate(ct_nonmut, ct_mut, k[0], beta), keys))

    max_rate = max(success_rates)
    key_list_with_max_rate = []
    for j, (k, _) in enumerate(keys):
        if success_rates[j] == max_rate:
            key_list_with_max_rate.append((k, success_rates[j]))
    keys = key_list_with_max_rate

    print("\n\n Iteration: {0}, keys_hex: {1}, keys: {2}, max key rate: {3}\n\n".format(i,
                                                                                        get_hex_tuple_0(
                                                                                            keys),
                                                                                        keys,
                                                                                        max_rate))
    # print("\n\n Iteration: {0}, keys_hex: {1}, keys: {2}, max key rate: {3}, success_rates: {4} \n\n".format(i,get_hex_tuple_0(keys),keys,max_rate,success_rates))

    # if there is only one key left, no sense in iterating further
    if len(keys) == 1:
        break



 Iteration: 0, keys_hex: [0x63, 0x6b, 0xe3, 0xeb, 0x863, 0x86b, 0x8e3, 0x8eb, 0x8063, 0x806b, 0x80e3, 0x80eb, 0x8863, 0x886b, 0x88e3, 0x88eb], keys: [(99, 28), (107, 28), (227, 28), (235, 28), (2147, 28), (2155, 28), (2275, 28), (2283, 28), (32867, 28), (32875, 28), (32995, 28), (33003, 28), (34915, 28), (34923, 28), (35043, 28), (35051, 28)], max key rate: 28




 Iteration: 1, keys_hex: [0x63, 0x6b, 0xe3, 0xeb, 0x863, 0x86b, 0x8e3, 0x8eb, 0x8063, 0x806b, 0x80e3, 0x80eb, 0x8863, 0x886b, 0x88e3, 0x88eb], keys: [(99, 17), (107, 17), (227, 17), (235, 17), (2147, 17), (2155, 17), (2275, 17), (2283, 17), (32867, 17), (32875, 17), (32995, 17), (33003, 17), (34915, 17), (34923, 17), (35043, 17), (35051, 17)], max key rate: 17




 Iteration: 2, keys_hex: [0x63, 0x6b, 0xe3, 0xeb, 0x863, 0x86b, 0x8e3, 0x8eb, 0x8063, 0x806b, 0x80e3, 0x80eb, 0x8863, 0x886b, 0x88e3, 0x88eb], keys: [(99, 7), (107, 7), (227, 7), (235, 7), (2147, 7), (2155, 7), (2275, 7), (2283, 7), (32867, 7), (32875, 7), (32995,

## Результат атаки:
* k_7 = 0x86b;
* кількість шт потрібна для знаходження -- 5 * 70_000 = 350000 шт ШТ.

## Висновки
За допомогою реалізації практикуму із блокових шифрів по знаходженню останньго раундового ключа для шифру Хейса дізналися на практиці, як проводяться атаки даного штибу для більших криптосистем.